# Data collapse

In [1]:
import os
import glob
# Our numerical workhorses
import numpy as np
import pandas as pd
import scipy

import mwc_induction_utils as mwc

# Import matplotlib stuff for plotting
import matplotlib.pyplot as plt
import matplotlib.cm as cm

# Seaborn, useful for graphics
import seaborn as sns

# favorite Seaborn settings for notebooks
rc={'lines.linewidth': 2, 
    'axes.labelsize' : 16, 
    'axes.titlesize' : 18,
    'axes.facecolor' : 'F4F3F6',
    'axes.edgecolor' : '000000',
    'axes.linewidth' : 1.2,
    'xtick.labelsize' : 13,
    'ytick.labelsize' : 13,
    'grid.linestyle' : ':',
    'grid.color' : 'a6a6a6'}
sns.set_context('notebook', rc=rc)
sns.set_style('darkgrid', rc=rc)
sns.set_palette("deep", color_codes=True)

# Magic function to make matplotlib inline; other style specs must come AFTER
%matplotlib inline

# This enables SVG graphics inline (only use with static plots (non-Bokeh))
%config InlineBackend.figure_format = 'svg'

## The Bohr parameter.

For the *lac* repressor we know that the fold-change in gene expression is given by
\begin{equation}
\text{fold-change}(C) = \left( 1 + \frac{2R}{N_{NS}} \cdot p_{act}(C) \cdot e^{-\beta \Delta \epsilon_r} \right)^{-1},
\end{equation}
where $R$ is the number of repressors, $N_{NS}$ is the number of non-specific sites to which the repressor can bind, $\Delta \epsilon_r$ is the binding energy of the repressor to the DNA binding site, $\beta$ is the inverse Boltzmann constant times the temperature, and $p_{act}$ is the probability of the repressor being active given a concentration of inducer $C$.

The functional form of this $p_{act}$ is given by the MWC model as
\begin{equation}
p_{act}(C) = \frac{\left( 1 + \frac{C}{K_A} \right)^2}{\left( 1 + \frac{C}{K_A} \right)^2 + e^{-\beta \Delta \epsilon}\left( 1 + \frac{C}{K_I} \right)^2},
\end{equation}
where $K_A$ and $K_I$ are the dissociation constants of the repressor in the active and inactive state respectively and $\Delta \epsilon$ is the energetic difference between these two states. For the specific case of the *lac* repressor we have that $K_A \geq K_I$ and $\Delta \epsilon \geq 0$.

To isolate the effect of the thermodynamic parameters we can re-write the fold-change equation as a Fermi potential, i.e.
\begin{equation}
\text{fold-change} = \frac{1}{1 + e^{-\beta F(C)}},
\end{equation}
where we have introduced the *Bohr parameter*,
\begin{equation}
F(C) \equiv -k_BT \log \left( \frac{\left( 1 + \frac{C}{K_A} \right)^2}{\left( 1 + \frac{C}{K_A} \right)^2 + e^{-\beta \Delta \epsilon}\left( 1 + \frac{C}{K_I} \right)^2} \cdot \frac{2R}{N_{NS}}e^{-\beta \Delta \epsilon_r} \right).
\end{equation}

Let's now go ahead and fit 